# Wrapping C++ coordinate descent function in Python with Cython and efficiency comparison

Finally successful @ Apr 21, 2016

## Import neccessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Code

### Naive Python

In [2]:
def nv_py_regular_linreg(X, y, beta_0, alpha, L1_ratio, max_iter=1000, tol=0.0001):
    def S(z, gamma):
        if np.abs(z) - gamma > 0:
            return np.sign(z)*(np.abs(z)-gamma)
        else:
            return 0
    N, p = X.shape
    beta = beta_0.copy()
    b_new = np.zeros(p)
    for itr in range(max_iter):
        for j in range(p):
            b_new[j] = S(1/N * np.dot(X[:,j], (y - (np.dot(X,beta) - np.dot(X[:,j], beta[j]))))
                     , alpha*L1_ratio)/(1+alpha*(1-L1_ratio))
        beta = b_new
    return beta            

### Python with ProcessPoolExecutor

Actually, in this particular problem, we do not see any neccesity to use multi-processing. Because the only parallelizable part is to compute the each feature vector related quantities, given features dimension will not be as large as 10000, thus the efficiency would not be improved much.

While, in fact, the efficiency may be dragged down: the time to set ProcessPool cannot compensate the little gains from parallel computing. The more outer loop (iteration), the more obvious the dragging is.

In [3]:
def deco(j, X, y, N, p, beta, alpha, L1_ratio):
    
    def S(z, gamma):
        if np.abs(z) - gamma > 0:
            return np.sign(z)*(np.abs(z)-gamma)
        else:
            return 0
        
    return S(1./N*np.dot(X[:,j], (y - (np.dot(X,beta) - np.dot(X[:,j], beta[j]))))
                     , alpha*L1_ratio)/(1+alpha*(1-L1_ratio))


def mp_py_regular_linreg(X, y, beta_0, alpha, L1_ratio, max_iter=50, tol=0.0001):
    from itertools import repeat
    from concurrent.futures import ProcessPoolExecutor
    
    N, p = X.shape
    beta = beta_0.copy()
    b_new = np.zeros(p)
    
    for itr in range(max_iter):
        with ProcessPoolExecutor(max_workers=8) as pool:
            b_new = np.array(list(pool.map(deco, [j for j in range(p)], repeat(X), repeat(y), repeat(N),
                                           repeat(p), repeat(beta), repeat(alpha), repeat(L1_ratio))))
        beta = b_new
    return beta  

### Python with C++ wrapped by Cython

#### wrting .hpp, .cpp, .pxd, .pyx files and config setup.py

In [4]:
%%file cpp_regularized_linreg.hpp
double estimated_y_i(double *x, int N, int p, double *beta, int i);
double compute_intermediate_b(double *x, int m, int n, double *y, double *beta, int j);
double S(double z, double gamma);    
double* coord_desc(double *x, double *y, int num_samples, int num_features, double *beta, 
                   double alpha, double L1_ratio, int max_iter, double tol);

Overwriting cpp_regularized_linreg.hpp


In [5]:
%%file cpp_regularized_linreg.cpp
#include <iostream>

using namespace std;

double estimated_y_i(double *x, int N, int p, double *beta, int i) {
	double s = 0;
	int l;
	for(l = 0; l < p; l++) {
		s += x[i*p + l] * beta[l];
	}
	return s;
}


double compute_intermediate_b(double *x, int m, int n, double *y, double *beta, int j) {
	double res = 0;
	int i;
	
	for(i = 0; i < m; i++) {
		res += x[i*n + j] * (y[i] - estimated_y_i(x, m, n, beta, i));
	}
	return res;
}

double S(double z, double gamma) {
	if(z >= 0) {
		if (z - gamma > 0) {
			return z - gamma;
		}
		else {
			return 0;
		}
	}
	else {
		if (-z - gamma > 0) {
			return z + gamma;
		}
		else {
			return 0;
		}
	}
}


double* coord_desc(double *x, double *y, int num_samples, int num_features, double *beta, 
                   double alpha, double L1_ratio, int max_iter=1000, double tol=0.0001) {
    
	int itr, j, i, 
    N = num_samples, 
    p = num_features;
    
	double bb;
    double b[p];
    
    double beta_new[p];
    
    double res;
    
    for(j = 0; j < p; j++){
        b[j] = beta[j];
    }
    
	for(itr = 0; itr < max_iter; itr++) {
		for(j = 0; j < p; j++) {
		       bb = S(b[j] + compute_intermediate_b(x, N, p, y, b, j) / N, alpha*L1_ratio) / (1. + alpha * (1 - L1_ratio));
		       beta_new[j] = bb;
		}
        
        for(j = 0; j < p; j++){
            b[j] = beta_new[j];
        } 
  
	}
    
    
    
    
    for(j = 0; j < p; j++){
            beta[j] = b[j];
        }
    
	return beta;
}



Overwriting cpp_regularized_linreg.cpp


In [6]:
%%file cy_regularized_linreg.pxd

cdef extern from "cpp_regularized_linreg.hpp":
    double* coord_desc(double *x, double *y, int num_samples, int num_features, double *beta, 
                       double alpha, double L1_ratio, int max_iter, double tol)

Overwriting cy_regularized_linreg.pxd


In [7]:
%%file cy_regularized_linreg.pyx
# distutils: language = c++
# distutils: sources = cpp_regularized_linreg.cpp

cimport cy_regularized_linreg

def py_regularized_linreg(double[::1] data_x, double[::1] data_y, int num_samples, int num_features, 
                          double[::1] beta, double alpha, double L1_ratio, int max_iter, double tol):
    
    cy_regularized_linreg.coord_desc(&data_x[0], &data_y[0], num_samples, num_features, 
                                                 &beta[0], alpha, L1_ratio, max_iter, tol)

Overwriting cy_regularized_linreg.pyx


In [8]:
%%file setup.py

from distutils.core import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext = Extension("cy_regularized_linreg",
                sources=["cy_regularized_linreg.pyx","cpp_regularized_linreg.cpp"],
                libraries=["m"],
                language=["c++"],
                extra_compile_args=["-std=c++11","-static"])

setup(name = "cy_regul_linreg",
      ext_modules = cythonize(ext))

Overwriting setup.py


#### compile

In [9]:
! python setup.py clean
! python setup.py -q build_ext --inplace

Compiling cy_regularized_linreg.pyx because it changed.
Cythonizing cy_regularized_linreg.pyx
running clean
removing 'build/temp.linux-x86_64-3.4' (and everything under it)
removing 'build'
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++ [enabled by default]
cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++ [enabled by default]
cpp_regularized_linreg.cpp: In function 'double* coord_desc(double*, double*, int, int, double*, double, double, int, double)':
cpp_regularized_linreg.cpp:48:14: warning: unused variable 'i' [-Wunused-variable]
  int itr, j, i, 
              ^
cpp_regularized_linreg.cpp:57:12: warning: unused variable 'res' [-Wunused-variable]
     double res;
            ^


## Justify solutions with data and scikit-learn

In [11]:
from sklearn import datasets
from sklearn.preprocessing import scale

### choose randomly generated large scale sparse data

In [12]:
## generate data
np.random.seed(42)

n_samples, n_features = 5000, 20
data_X = np.random.randn(n_samples, n_features)
coef = 3 * np.random.randn(n_features)
inds = np.arange(n_features)
np.random.shuffle(inds)
coef[inds[10:]] = 0  # sparsify coef
data_y = np.dot(data_X, coef)
# add noise
data_y += 0.02 * np.random.normal((n_samples,))

## standardize data
X, y = scale(data_X, axis=0), scale(data_y)
print(X.shape, y.shape)

## resahpe data into 1D
X_resh = X.reshape((n_samples*n_features,))
num_samples, num_features = n_samples, n_features

(5000, 20) (5000,)


### choose sklearn data

In [13]:
ds = datasets.load_diabetes()
data_X = ds.data
data_y = ds.target
num_samples, num_features = data_X.shape
print(data_X.shape, data_y.shape)

## standardize data
X, y = scale(data_X, axis=0), scale(data_y)

## resahpe data into 1D
X_resh = X.reshape((num_samples*num_features,))

(442, 10) (442,)


### scikit-learn results

#### lasso
$\alpha = 0.5$

In [14]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.5)
lasso.fit(X, y)
print(lasso.coef_)

[ 0.          0.          0.07123662  0.          0.          0.         -0.
  0.          0.03410059  0.        ]


#### elastic net
$\alpha = 1$,

$L1\_ratio = 0.5$

In [15]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=1, l1_ratio=0.5)
enet.fit(X, y)
print(enet.coef_)

[ 0.        0.        0.048895  0.        0.        0.       -0.        0.
  0.029379  0.      ]


### our results: Python with wrapped C++ by Cython

#### lasso
$\alpha = 0.5$

In [16]:
import cy_regularized_linreg
beta = np.zeros(num_features)
cy_regularized_linreg.py_regularized_linreg(X_resh, y, num_samples, num_features, beta, 0.5, 1, 1000, 0.001)
print(beta)

[ 0.          0.          0.07123567  0.          0.          0.          0.
  0.          0.03410101  0.        ]


#### elastic net
$\alpha = 1$,

$L1\_ratio = 0.5$

In [17]:
beta = np.zeros(num_features)
cy_regularized_linreg.py_regularized_linreg(X_resh, y, num_samples, num_features, beta, 1, 0.5, 1000, 0.001)
print(beta)

[ 0.          0.          0.04889496  0.          0.          0.          0.
  0.          0.02937901  0.        ]


## Comparing efficiency

### lasso

#### naive python

In [18]:
%timeit nv_py_regular_linreg(X, y, np.zeros(num_features), alpha=0.5, L1_ratio=1)

10 loops, best of 3: 105 ms per loop


#### python with ProcessPoolExecutor

In [19]:
%timeit mp_py_regular_linreg(X, y, np.zeros(num_features), alpha=0.5, L1_ratio=1)

1 loop, best of 3: 2.24 s per loop


#### scikit-learn

In [20]:
%timeit lasso.fit(X, y)

1000 loops, best of 3: 309 µs per loop


#### python with c++

In [21]:
%time cy_regularized_linreg.py_regularized_linreg(X.reshape((np.prod(X.shape),)), y, num_samples, num_features, \
                                                  np.zeros(num_features), 1, 0.5, 1000, 0.001)

CPU times: user 48.2 ms, sys: 0 ns, total: 48.2 ms
Wall time: 47.7 ms


### elastic net

#### naive python

In [22]:
%timeit nv_py_regular_linreg(X, y, np.zeros(num_features), alpha=0.5, L1_ratio=1)

10 loops, best of 3: 105 ms per loop


#### python with ProcessPoolExecutor

In [23]:
%timeit mp_py_regular_linreg(X, y, np.zeros(num_features), alpha=1, L1_ratio=0.5)

1 loop, best of 3: 2.25 s per loop


#### scikit-learn

In [24]:
%timeit enet.fit(X, y)

1000 loops, best of 3: 308 µs per loop


#### python with c++

In [25]:
%timeit cy_regularized_linreg.py_regularized_linreg(X.reshape((np.prod(X.shape),)), y, \
                                            num_samples, num_features, np.zeros(num_features), 1, 0.5, 1000, 0.001)

10 loops, best of 3: 46.5 ms per loop


## Conclusion

Our Python code with C++ wrapped by Cython obviously improves the efficiency by roughly 50%.

The ProcessPool just lowers the computing speed.